In [ ]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
cwd

In [ ]:
#https://en.wikipedia.org/wiki/Non-return-to-zero
def NRZ_maker(values):
    NRZ = []
    state = None
    prior = values[0]

    for bit in values:
        if bit == prior:
            state = 1.0
        else:
            state = 0.0
        NRZ.append(state)
    return NRZ

In [ ]:
test_array = np.array([6, 6, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12])
test_df = pd.DataFrame({'values': test_array})
values = test_df['values'].tolist()

NRZ = []
state = 0
prior = values[0]
for bit in values:
    if bit == prior:
        state = 1
    else:
        state = 0
    NRZ.append(state)
#NRZ

test_df['NRZ'] = NRZ
test_df.head()

In [ ]:
values = test_df['values'].tolist()
test_df['NRZ2'] = NRZ_maker(values)
test_df.head()

In [ ]:
PATH = '/data22/bde'
netflix_StrangerThings = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/Season3-StrangerThings/'
netflix_animated_RickandMorty = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/animated-RickandMorty/'
amazon_TheExpanse = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/Season3-TheExpanse/'
amazon_animated_AdventureTime = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/animated-AdventureTime/'
download = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
driving_streaming = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving-Streaming/'
driving = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)
    #df.info()

    values = df['CellID'].tolist()
    df['NRZ'] = NRZ_maker(values)
    df['NRZ'] = df['NRZ'].astype('int32')

    nrz_one_hot = pd.get_dummies(df['NRZ'], prefix='NRZ', dtype=int)
    #nrz_one_hot.info()
    df['NRZ_0'] = nrz_one_hot['NRZ_0']
    df['NRZ_1'] = nrz_one_hot['NRZ_1']

    df['CellIDChange'] = (df['CellID'] != df['CellID'].shift()).astype('int32')

    cellid_one_hot = pd.get_dummies(df['CellIDChange'], prefix='CellIDChange', dtype=int)
    #cellid_one_hot.info()
    df['CellIDChange_0'] = cellid_one_hot['CellIDChange_0']
    df['CellIDChange_1'] = cellid_one_hot['CellIDChange_1']
    
    df = df[['Timestamp', 'CellID', 'NRZ', 'NRZ_0', 'NRZ_1', 'CellIDChange', 'CellIDChange_0', 'CellIDChange_1', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'DL_bitrate', 'UL_bitrate', 'Speed']]
    df = df.replace('-', np.nan)
    df = df.replace('2147483647.0', 0)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    #df.fillna(method='ffill', inplace=True)
    df.fillna(0, inplace=True)

    df['CellID'] = pd.to_numeric(df['CellID'])
    df['NRZ'] = pd.to_numeric(df['NRZ'])
    df['NRZ'] = pd.to_numeric(df['NRZ_0'])
    df['NRZ'] = pd.to_numeric(df['NRZ_1'])

    #df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='integer')
    df['RSRP'] = pd.to_numeric(df['RSRP'])
    
    df['RSRQ'] = pd.to_numeric(df['RSRQ'])
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='integer')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='integer')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='integer')
    df['NRxRSRP'] = pd.to_numeric(df['NRxRSRP'], downcast='integer')
    df['NRxRSRQ'] = pd.to_numeric(df['NRxRSRQ'], downcast='integer')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'])
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'])

    #df['RSRPoffset'] = df['RSRP'] - df['NRxRSRP']
    #df['RSRQoffset'] = df['RSRQ'] - df['NRxRSRQ']

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    #print(df.head())
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

In [ ]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
df = df[['date', 'CellID', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'UL_bitrate', 'DL_bitrate']]
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def multi_corr(feature):
    fig, ax = plt.subplots(1,3, figsize=(20, 6))

    # when the Pearson and Spearman values are not much different, 
    # our data tends to not have extreme values (outliers)
    corr1 = df.corr('pearson')[[feature]].sort_values(by=feature, ascending=False)
    corr2 = df.corr('spearman')[[feature]].sort_values(by=feature, ascending=False)

    corr3 = df.corr('kendall')[[feature]].sort_values(by=feature, ascending=False)
    #ordinal correlation (Spearman & Kendall Tau)

    sns.heatmap(corr1, ax=ax[0], annot=True)
    sns.heatmap(corr2, ax=ax[1], annot=True)
    sns.heatmap(corr3, ax=ax[2], annot=True)

In [ ]:
multi_corr('DL_bitrate')

In [ ]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)

In [ ]:
# number of changes in values
(df != df.shift(axis=0)).sum(axis=0)